In [38]:
# Remember to eliminate first 4 rows on gdp.csv 
#(https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv)
#provenienza dataset  Food and Agricultural Organization of United Nations
#https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
import pandas as pd
import numpy as np
crop = pd.read_csv("crop1.csv")
live = pd.read_csv("live1.csv")
pop = pd.read_csv("pop1.csv")
gdp = pd.read_csv("gdp.csv")
ccc = pd.read_csv("countries_codes_and_coordinates.csv")

In [39]:
#for beehives it may mean the number of hives and not bees
live['Value'] = np.where(live['Unit'] == '1000 Head',live['Value'] * 1000,live['Value'])
live["Unit"] = 'Head'

In [40]:
crop = crop.append(live)

In [41]:
crop = crop[crop.Area != "Australia and New Zealand"] #Australia and New Zealand = Australia+New Zealand
crop = crop[crop.Area != "China, Hong Kong SAR"]#China=China Hong Kong+Macao+Taiwan+mainland
crop = crop[crop.Area != "China, Macao SAR"]
crop = crop[crop.Area != "China, Taiwan Province of"]
crop = crop[crop.Area != "China, mainland"]

crop['Area'].replace(['Bolivia (Plurinational State of)'], ['Bolivia'],inplace=True)
crop['Area'].replace(['Cabo Verde'], ['Cape Verde'],inplace=True)
crop['Area'].replace(['Czechia'], ['Czech Republic'],inplace=True)
crop['Area'].replace(['Ethiopia PDR'], ['Ethiopia'],inplace=True)
crop['Area'].replace(['French Guyana'], ['French Guiana'],inplace=True)
crop['Area'].replace(['Democratic Republic of the Congo'], ['Congo, the Democratic Republic of the'],inplace=True)
crop['Area'].replace(['Iran (Islamic Republic of)'], ['Iran, Islamic Republic of'],inplace=True)
crop['Area'].replace(["Democratic People's Republic of Korea"], ["Korea, Democratic People's Republic of"],inplace=True)
crop['Area'].replace(['Republic of Korea'], ['South Korea'],inplace=True)
crop['Area'].replace(['Republic of Moldova'], ['Moldova, Republic of'],inplace=True)
crop['Area'].replace(['North Macedonia'], ['Macedonia, the former Yugoslav Republic of'],inplace=True)
crop['Area'].replace(['Palestine'], ['Palestinian Territory, Occupied'],inplace=True)
crop['Area'].replace(['Sudan (former)'], ['Sudan'],inplace=True)
crop['Area'].replace(['USSR'], ['Russian Federation'],inplace=True)
crop['Area'].replace(['United Kingdom of Great Britain and Northern Ireland'], ['United Kingdom'],inplace=True)
crop['Area'].replace(['United States of America'], ['United States'],inplace=True)
crop['Area'].replace(['Venezuela (Bolivarian Republic of)'], ['Venezuela'],inplace=True)

In [42]:
#Belgium-Luxembourg fino al 2000, poi Belgium e Luxembourg separati
#Czechoslovakia fino al 1992, Czechia e Slovakia  dal 1993
#Serbia and Montenegro fino al 2005, poi Serbia e Montenegro 

In [43]:
ccc.drop(["Alpha-2 code","Numeric code","Latitude (average)","Longitude (average)"],axis=1,inplace=True)
ccc["Alpha-3 code"]=ccc["Alpha-3 code"].apply(lambda x: ''.join(e for e in x if e.isalnum()))

In [44]:
crop = crop.merge(ccc,how="left",left_on=["Area"],right_on=["Country"])

In [45]:
gdp.drop(["Indicator Code","Indicator Name"],axis=1,inplace=True)
gdp = gdp.melt(id_vars=["Country Name","Country Code"],var_name="Year",value_name="GDP")
gdp = gdp.astype({"Year" : "int64"})

In [46]:
c = [0.0]
c.extend(list(crop["Value"][:-1].values))
crop["Previous_value"] = c
crop.loc[crop[["Area","Item","Element"]].drop_duplicates().index,"Previous_value"] = 0

In [47]:
crop["Increase"] = 100*(crop["Value"]-crop["Previous_value"])/crop["Previous_value"]
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
crop["Increase"] = round(crop["Increase"], 2)

In [48]:
crop = crop.merge(gdp,how="left",left_on=["Year","Alpha-3 code"],right_on=["Year","Country Code"])
crop.drop(["Alpha-3 code","Country Name"],axis=1,inplace=True)
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [49]:
c = [0.0]
c.extend(list(crop["GDP"][:-1].values))
crop["Previous_GDP"] = c
crop.loc[crop[["Area","Item","Element"]].drop_duplicates().index,"Previous_GDP"] = 0

In [50]:
crop["Increase_GDP"] = 100*(crop["GDP"]-crop["Previous_GDP"])/crop["Previous_GDP"]
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
crop["Increase_GDP"] = round(crop["Increase_GDP"], 2)

In [51]:
pop = pop.melt(id_vars=["Country Name","Country Code"],var_name="Year",value_name="Pop")
pop = pop.astype({"Year" : "int64"})
crop = crop.merge(pop,how="left",left_on=["Area","Year"],right_on=["Country Name","Year"])
crop.drop(["Country Name","Country","Country Code_y"],axis=1,inplace=True)
crop.rename(columns={"Country Code_x":"Country Code"}, inplace=True)
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [52]:
c = [0.0]
c.extend(list(crop["Pop"][:-1].values))
crop["Previous_Pop"] = c
crop.loc[crop[["Area","Item","Element"]].drop_duplicates().index,"Previous_Pop"] = 0
crop["Increase_Pop"] = 100*(crop["Pop"]-crop["Previous_Pop"])/crop["Previous_Pop"]
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
crop["Increase_Pop"] = round(crop["Increase_Pop"], 2)

In [53]:
fruit_vegetables = ['Apples','Apricots','Artichokes','Asparagus','Avocados','Bananas','Berries nes',
                   'Cabbages and other brassicas','Carrots and turnips','Castor oil seed',
                    'Cauliflowers and broccoli','Cherries','Chillies and peppers, dry',
                    'Chillies and peppers, green','Coconuts','Coffee, green','Cow peas, dry','Cucumbers and gherkins',
                   'Dates','Eggplants (aubergines)','Figs','Fruit, citrus nes','Fruit, fresh nes','Fruit, stone nes',
                    'Fruit, tropical fresh nes','Garlic','Grapefruit (inc. pomelos)','Grapes','Hops','Kiwi fruit',
                    'Leeks, other alliaceous vegetables','Lemons and limes','Lettuce and chicory',
                    'Mangoes, mangosteens, guavas','Melons, other (inc.cantaloupes)','Melonseed','Onions, dry',
                    'Onions, shallots, green','Oranges','Papayas','Peaches and nectarines','Pears','Peas, dry',
                    'Peas, green','Pepper (piper spp.)','Persimmons','Pineapples','Plums and sloes',
                   'Pumpkins, squash and gourds','Quinces','Spinach','Strawberries','Sugar beet',
                    'Tangerines, mandarins, clementines, satsumas','Taro (cocoyam)','Tomatoes',
                    'Vegetables, fresh nes','Vegetables, leguminous nes','Watermelons']
tubers = ['Cassava','Potatoes','Sweet potatoes']
cereals = ['barley','Buckwheat','Cereals nes','Maize','Maize, green','Millet','Oats','Rice, paddy',
           'Roots and tubers nes','Rye','Sorghum','Triticale','Wheat'] 
herbs = ['Anise, badian, fennel, coriander','Cinnamon (cannella)','Cloves','Ginger','Linseed','Mustard seed',
        'Pepper (piper spp.)','Spices nes','Sugar cane','Tea','Tobacco, unmanufactured','Vanilla']
nuts = ['Almonds, with shell','Cashew nuts, with shell','Chestnut','Groundnuts, with shell','Hazelnuts, with shell',
       'Nuts nes','Pistachios','Tung nuts','Walnuts, with shell']
fibres = ['Bastfibres, other','Flax fibre and tow','Hemp tow waste','Jute','Ramie','Sisal','Tallowtree seed']
legumes = ['Beans, dry','Beans, green','Broad beans, horse beans, dry','Chick peas','Lentils','Pulses nes',
          'Soybeans']
mushrooms = ['Mushrooms and truffles']
oils = ['Oil palm fruit','Oilseeds nes','Rapeseed','Sunflower seed']
other = ['Rubber, natural','Safflower seed','Seed cotton','Sesame seed']
crops=[fruit_vegetables,tubers,cereals,herbs,nuts,fibres,legumes,mushrooms,oils,other]
crops_type = ["fruit_vegetables","tubers","cereals","herbs","nuts","fibres","legumes","mushrooms","oils","other"]
def type_crop(row):
    for x in crops:
        if row["Item"] in x:
            return crops_type[crops.index(x)]
    return "Live"
crop['type'] = crop.apply(type_crop, axis=1)

In [54]:
aggregated_countries = ['Africa','Americas','Asia','Australia and New Zealand','Belgium-Luxembourg','Central America',
                        'Central Asia','China, Hong Kong SAR','China, Macao SAR','China, Taiwan Province of',
                        'China, mainland','Eastern Africa', 'Eastern Asia','Eastern Europe','Europe',
                        'European Union (27)','Land Locked Developing Countries','Least Developed Countries',
                        'Low Income Food Deficit Countries','Middle Africa', 'Net Food Importing Developing Countries',
                        'Northern Africa','Northern America','Northern Europe','Oceania',
                        'Small Island Developing States','South Africa','South America','South-eastern Asia',
                        'Southern Africa','Southern Asia','Southern Europe','Western Africa','Western Asia',
                        'Western Europe','World']
crop["Agg"] = crop['Area'].apply(lambda x: 1 if x in aggregated_countries else 0)

In [55]:
glt = pd.read_csv("GlobalLandTemperaturesByCountry.csv")
glt=glt[(glt["dt"]>"1961")&(glt["dt"]<"2013")]
glt["aggdt"]=glt['dt'].apply(lambda x: x.split('-')[0])

In [56]:
glt=glt.groupby(["Country","aggdt"],group_keys=False).sum()/12
glt = glt.reset_index(drop=False)

In [57]:
glt['Country'].replace(['Bosnia And Herzegovina'], ['Bosnia and Herzegovina'],inplace=True)
glt['Country'].replace(['Congo (Democratic Republic Of The)'], ['Congo, the Democratic Republic of the'],inplace=True)
glt['Country'].replace(['Denmark (Europe)'], ['Denmark'],inplace=True)
glt['Country'].replace(['France (Europe)'], ['France'],inplace=True)
glt['Country'].replace(['Iran'], ['Iran, Islamic Republic of'],inplace=True)
glt['Country'].replace(['North Korea'], ["Korea, Democratic People's Republic of"],inplace=True)
glt['Country'].replace(['Macedonia'], ["Macedonia, the former Yugoslav Republic of"],inplace=True)
glt['Country'].replace(['Moldova'], ["Moldova, Republic of"],inplace=True)
glt['Country'].replace(['Netherlands (Europe)'], ["Netherlands"],inplace=True)
glt['Country'].replace(['Palestina'], ["Palestinian Territory, Occupied"],inplace=True)
glt['Country'].replace(['Russia'], ["Russian Federation"],inplace=True)
glt['Country'].replace(['United Kingdom (Europe)'], ["United Kingdom"],inplace=True)
glt['Country'].replace(['Vietnam'], ["Viet Nam"],inplace=True)

In [58]:
glt = glt.merge(ccc,how="left",left_on=["Country"],right_on=["Country"])

In [59]:
glt = glt.astype({"aggdt" : "int64"})
glt.drop(["AverageTemperatureUncertainty"],axis=1,inplace=True)
crop = crop.merge(glt,how="left",left_on=["Year","Country Code"],right_on=["aggdt","Alpha-3 code"])
crop.drop(["aggdt","Country","Alpha-3 code"],axis=1,inplace=True)
crop.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [61]:
crop = crop[(crop["Element"]=="Production")|(crop["Element"]=="Stocks")]

In [62]:
crop.to_csv('farming.csv',index=False)